In [1]:
import matplotlib.pyplot as plt
import torch

from src.core.environment.data_loader import DataLoader
from src.core.model.encoders.multimodal_encoder import MultimodalEncoder

/Users/crinstaniev/opt/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
a = DataLoader(train=True)

75
['meridian-art-blocks-playground', 'cyberkongz-babies', 'cryptodickbutts', 'veefriends-series-2', 'Decentraland', 'cyberkongz-vx', 'killabears', 'curio-cards', 'meebits', 'memories-of-qilin-art-blocks-curated', 'the-potatoz', 'degods', 'hv-mtl-nft', 'cyberkongz-genesis', 'terraforms-by-mathcastles', 'onchainmonkey', 'doodles', 'murakami-flowers', 'pudgy-penguins', 'youtherealmvp', 'live-like-a-cat-nekonoyouniikiru', 'world-of-women', 'cryptoninja-partners', 'renga', 'veefriends', 'damien-hirst-the-currency', 'io-imaginary-ones', 'friendship-bracelets-by-alexis-andre', 'nft-worlds', 'eulerbeats-genesis-original', 'the_sandbox', 'the-singularity', 'fluf-world', 'etheria', 'neo-tokyo-part-4-land-deeds', 'cryptopunks', 'kanpai-pandas', 'nakamigos', 'qql-mint-pass', 'invisible-friends', 'sappy-seals', 'elevated-deconstructions-art-blocks-curated', 'genesis-braindrops', 'gazers', 'opepen-edition-nft', 'v1-cryptopunks-wrapped', 'ringers-art-blocks-curated', 'milady', 'space-doodles', 'forg

In [3]:
text, image, time_series = a.load_collection_features(0)

In [4]:
time_series.shape

torch.Size([1, 16, 5])

In [5]:
image.shape

torch.Size([1, 3, 224, 224])

In [9]:
text

["Meridian - Art Blocks Playground is an NFT collectible created by Matt DesLauriers that was released on 9-28-2021. The project consists of 1,000 unique digital items living on the Ethereum blockchain. We categorize it as a art project and it's part of the Art Blocks general collection and the Playground subcollection."]

In [14]:
text = [
        'This is a sample text to test the MultimodalEncoder module.'
        for _ in range(1)
    ]  # (batch_size, x)

In [15]:
text

['This is a sample text to test the MultimodalEncoder module.']

In [17]:
time_series

tensor([[[7.0000e+00, 2.8305e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [7.0000e+00, 2.8690e+04, 2.0000e+00, 1.6990e+01, 6.9634e+04],
         [6.9900e+00, 3.0046e+04, 3.0000e+00, 2.4990e+01, 1.0742e+05],
         [6.9900e+00, 3.1101e+04, 2.0000e+00, 1.8050e+01, 8.0312e+04],
         [6.9690e+00, 3.2311e+04, 3.0000e+00, 2.6000e+01, 1.2055e+05],
         [6.9690e+00, 3.1966e+04, 2.0000e+00, 8.6969e+01, 3.9892e+05],
         [6.9500e+00, 3.1376e+04, 2.0000e+00, 4.9000e+01, 2.2121e+05],
         [6.2500e+00, 2.6352e+04, 3.0000e+00, 2.0000e+01, 8.4326e+04],
         [5.7500e+00, 2.3708e+04, 2.0000e+00, 1.3750e+01, 5.6693e+04],
         [6.5000e+00, 2.7321e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.0000e+00, 2.6125e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.0000e+00, 2.5861e+04, 2.0000e+00, 1.3890e+01, 5.9869e+04],
         [6.0000e+00, 2.6646e+04, 2.0000e+00, 1.5000e+01, 6.6614e+04],
         [6.0000e+00, 2.4920e+04, 3.0000e+00, 1.9000e+01, 7.8913e+04],
      

In [13]:
output = encoder(time_series, image, text)

size: torch.Size([1, 16, 5])


RuntimeError: mat1 and mat2 must have the same dtype